<a href="https://colab.research.google.com/github/anuragkj/GDSC_ML_Training_Project_1/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib','inline')
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score, learning_curve
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

import xgboost as xgb
import lightgbm as lgb

sns.set(style='white', context='notebook', palette='deep')

In [13]:
import numpy as np
import pandas as pd

In [14]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
invoice_test = pd.read_csv('/content/drive/MyDrive/Project_1_data/Test/client_test.csv')
invoice_train = pd.read_csv('/content/drive/MyDrive/Project_1_data/Train/invoice_train.csv')
client_test = pd.read_csv('/content/drive/MyDrive/Project_1_data/Test/client_test.csv')
client_train = pd.read_csv('/content/drive/MyDrive/Project_1_data/Train/client_train.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/Project_1_data/SampleSubmission.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
